In [ ]:
import matplotlib.pyplot as plt
import DMK_go_coude as Fns
import numpy as np
import os, readcol
import scipy.optimize as optim
import scipy.interpolate as interp
import pickle

from astropy.io import fits 
from mpfit import mpfit
from scipy import signal

In [ ]:
dir = os.getenv("HOME") + '/Research/YMG/coude_data/20140321/'
rdir = dir + 'reduction/'
codedir = os.getenv("HOME") + '/codes/coudereduction/'
# codedir = os.getenv("HOME") + '/Research/Codes/coudereduction/'

In [ ]:
os.chdir(dir)

In [ ]:
# DarkCurVal = 0.0

# InfoFile = 'headstrip.csv'
# FileInfo = readcol.readcol( InfoFile, fsep = ',', asRecArray = True )
# DarkCube = FileInfo.ExpTime * DarkCurVal

# BiasInds = np.where( FileInfo.Type == 'zero' )[0]
# FlatInds = np.where( FileInfo.Type == 'flat' )[0]
# ArcInds  = np.where( (FileInfo.Type == 'comp') & ( (FileInfo.Object == 'Thar') | (FileInfo.Object == 'THAR') | (FileInfo.Object == 'A') ) )[0]
# ObjInds  = np.where( (FileInfo.Type == 'object') & (FileInfo.Object != 'SolPort') & (FileInfo.Object != 'solar port') & (FileInfo.Object != 'solar_ort') )[0]

# CalsDone = True
# SuperBias, FlatField = Fns.Basic_Cals( FileInfo.File[BiasInds], FileInfo.File[FlatInds], CalsDone, rdir, plots = False )

# ShowBPM = False
# BPM = Fns.Make_BPM( SuperBias, FlatField, 99.9, ShowBPM )

# RdNoise  = FileInfo.rdn[ArcInds] / FileInfo.gain[ArcInds]
# DarkCur  = DarkCube[ArcInds] / FileInfo.gain[ArcInds]
# ArcCube, ArcSNR = Fns.Make_Cube( FileInfo.File[ArcInds], RdNoise, DarkCur, Bias = SuperBias )

# RdNoise  = FileInfo.rdn[ObjInds] / FileInfo.gain[ObjInds]
# DarkCur  = DarkCube[ObjInds] / FileInfo.gain[ObjInds]
# ObjCube, ObjSNR = Fns.Make_Cube( FileInfo.File[ObjInds], RdNoise, DarkCur, Bias = SuperBias, Flat = FlatField, BPM = BPM )

# OrderStart = -32
# TraceDone = True
# MedCut = 90.0
# MedTrace, FitTrace = Fns.Get_Trace( FlatField, ObjCube, OrderStart, MedCut, rdir, TraceDone, plots = False )

# wspec,sig_wspec = Fns.extractor( ArcCube,ArcSNR,FitTrace,quick=True,arc=True,nosub=True )

In [ ]:
spec       = pickle.load(open(rdir+'extracted_spec_oldway.pkl','rb'))
sig_spec   = pickle.load(open(rdir+'extracted_sigspec_oldway.pkl','rb'))
wspec      = pickle.load(open(rdir+'extracted_wspec_oldway.pkl','rb'))
sig_wspec  = pickle.load(open(rdir+'extracted_sigwspec_oldway.pkl','rb'))

wspec      = wspec[:,::-1,:]
sig_wspec  = sig_wspec[:,::-1,:]
spec       = spec[:,::-1,:]
sig_spec   = sig_spec[:,::-1,:]

In [ ]:
roughsol = pickle.load( open( codedir + 'prelim_wsol.pkl', 'rb' ) )
sols, params = Fns.Get_WavSol( wspec, roughsol, rdir, codedir, Orders = [8] )

In [ ]:
class Click_WavSol():
    '''
    A class for handling listening for user clicks on matplotlib plots and passing the 
    coordinates for use in the wavelength calibration.
    '''
    def __init__(self, obswav, obsarc, calwav, calarc, lines ):
        self.obswav   = obswav
        self.obsarc   = obsarc
        self.calwav   = calwav
        self.calarc   = calarc
        self.lines    = lines
        self.obsclick = np.array([])
        self.calclick = np.array([])
        
    def getcoord(self):

        plt.clf()
        plt.plot( self.calwav, self.calarc, 'r-' )
        plt.plot( self.obswav, self.obsarc, 'k-' )
        for line in self.lines:
            plt.axvline( x = line, color = 'b', ls = ':' )
        plt.xlim( np.min( self.obswav ) - 5.0, np.max( self.obswav ) + 5.0 )
        
        fig = plt.gcf()
        cid = fig.canvas.mpl_connect( 'key_press_event', self.__onclick__ )
        plt.show()
        
        return self.obsclick, self.calclick

    def __onclick__(self, click):
        print click.xdata
        if click.key == 'a':
            self.obsclick = np.append( self.obsclick, click.xdata )
            return self.obsclick
        elif click.key == 'b':
            self.calclick = np.append( self.calclick, click.xdata )
            return self.calclick
        else:
            print 'Please press a for an observed line and b for a calibrated line.\n'
            print 'No other key will do anything for ya.'